In [1]:
!pip install splinter
!pip install bs4

  Stored in directory: C:\Users\umnaz\AppData\Local\pip\Cache\wheels\fd\ff\26\2fc56897bcbe58908bbb2002c53affb516e5e6daa425aefdef
Successfully built splinter
  Stored in directory: C:\Users\umnaz\AppData\Local\pip\Cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time

In [11]:
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

# Visit the NASA mars news site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time = 1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

slide_elem = soup.select_one("ul.item_list li.slide")

slide_elem.find("div", class_= "content_title" )

<div class="content_title"><a href="/news/8663/nasas-perseverance-rover-will-look-at-mars-through-these-eyes/" target="_self">NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'</a></div>

In [12]:
news_title = slide_elem.find("div", class_ = "content_title").get_text()
news_title

"NASA's Perseverance Rover Will Look at Mars Through These 'Eyes'"

In [13]:
news_p = slide_elem.find("div", class_ = "article_teaser_body").get_text()
news_p

'A pair of zoomable cameras will help scientists and rover drivers with high-resolution color images.'

In [22]:
# JPL Space Images Features Images

executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

# Find the more info button and click that

browser.is_element_present_by_text('more info', wait_time =1)
more_info_elem = browser.links.find_by_partial_text("more info")
more_info_elem.click()

#Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, "html.parser")

# find the relative image url

img_url_rel = img_soup.select_one("figure.lede a img").get("src")
img_url_rel

img_url = f"https://www.jpl.nasa.gov{img_url_rel}"
img_url


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19036_hires.jpg'

In [26]:
# Mars weather


executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
time.sleep(5)

html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweets", "data-name": "Mars Weather"})

try:
    mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
    mars_weather
except AttributeError:
    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
mars_weather

    

'InSight sol 508 (2020-05-01) low -92.2ºC (-134.0ºF) high -2.4ºC (27.7ºF)\nwinds from the SW at 5.1 m/s (11.3 mph) gusting to 15.8 m/s (35.3 mph)\npressure at 6.80 hPa'

# Mars Hemisphere

In [35]:
executable_path = {"executable_path":"chromedriver.exe"}
browser = Browser('chrome', **executable_path, headless=False)


url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

hemisphere_image_urls = []
links = browser.find_by_css("a.product-item h3")

for i in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[i].click()
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere["img_url"]= sample_elem["href"]
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    hemisphere_image_urls.append(hemisphere)
    browser.back()
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [36]:
# Mars facts

import pandas as pd
df = pd.read_html("https://space-facts.com/mars/")[0]
df.columns = ["description", "value"]
df.set_index("description", inplace=True)
df


,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [37]:
df.to_html()
browser.quit()